#Save the model in 'spam text classifier.ipynb' and load it here

In [6]:
import joblib

In [7]:
model1 = joblib.load('/content/model.pkl')

In [8]:
model1.intercept_

array([-0.38588863])

In [9]:
pip install sentence_transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
from sentence_transformers import SentenceTransformer

In [11]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
q = input("enter the message then I will let you know if it is a spam : ")

enter the message then I will let you know if it is a spam : hello, how are you


In [13]:
X_bert = model.encode(q)

In [14]:
pred = model1.predict([X_bert])

In [15]:
if pred == "spam":
  print("This is a spam message")
else:
  print("This is not a spam message")

This is not a spam message


#Telegram polling loop

In [32]:
from datetime import datetime
import requests
import time

from google.colab import userdata
telegram_api_key = userdata.get('TELEGRAM_BERT_API_KEY')
url = f'https://api.telegram.org/bot{telegram_api_key}/'
#print(url)

In [33]:
offset = -1

while True:
  try:
    response = requests.get(f'{url}getUpdates?offset={offset}')
    data = response.json()["result"]
  except Exception as e:
    print(f'Exception caught: {e}')
    break


  if data:
    # message returned from Telegram getUpdates
    last_message = data[-1]
    chat_id = last_message['message']['chat']['id'] # chat_id is unique for every user (non-group chat)

    message_time = last_message['message']['date']
    current_time = int(time.time())

    # ignore old messages
    if (current_time - message_time) > 10:
      offset = last_message['update_id'] + 1
      print('ignored old message')
      continue  # wait for incoming messages again




    q = last_message['message']['text']


    # only begin a chat session if the command /start is received
    if q.lower() == '/start':

      r = requests.get(url + f'sendMessage?chat_id={chat_id}&text={"Send me some text: (or type /quit)"}')

      while True:
        last_update_id = last_message['update_id']
        offset = last_update_id + 1

        time.sleep(5)

        response = requests.get(f'{url}getUpdates?offset={offset}')
        data = response.json()["result"]

        if not data:
          continue

        last_message = data[-1]
        q = last_message['message']['text']

        # end the session if command /quit is received
        if q.lower() == "/quit":
          requests.get(url + f'sendMessage?chat_id={chat_id}&text=Good%20Bye!')
          break
        else:

          # show "typing..."
          send_action = url + f'sendChatAction?chat_id={chat_id}&action=typing'
          requests.get(send_action)

          # run classification
          X_bert = model.encode(q)
          pred = model1.predict([X_bert])
          if pred=="ham":
            result = "Not Spam"
          else:
            result = "Spam"

          send_url = url + f'sendMessage?parse_mode=markdown&chat_id={chat_id}&text={q} Result: {result}'
          requests.get(send_url)

          requests.get(url + f'sendMessage?parse_mode=markdown&chat_id={chat_id}&text=Send me some text: (or type /quit)')

    else:
      # last message was not /start (most likely the quit from last session), wait for command
      # setting the offset here effectively marks the last message as "confirmed"
      # and will not be returned again from getUpdates
      if offset == -1:
        offset = last_message['update_id'] + 1
        print(offset)
      time.sleep(3)
      continue


ignored old message


KeyboardInterrupt: 